This code produces the poset of all weak maps, given a fixed element set


In [17]:
using Oscar
using Combinatorics
n_elements = 7;
rk = 3;
db = Polymake.Polydb.get_db();
collection = db["Matroids.Small"];
query = Dict("N_ELEMENTS"=>n_elements);
results = Polymake.Polydb.find(collection, query);
oscar_matroids = [Matroid(pm) for pm in results];

In [18]:
all_matroids =Matroid[];
counter =0
ground_set = [i for i in 1:n_elements];
permuts = permutations(ground_set);
for m in oscar_matroids
    iso= [];
    for perm in permuts
        #println("current permutation is")
        #println(perm)
        current_b = Set();
        #println(current_b);

        for b in bases(m)
            #println("current base is" )
            #println(b)
            permuted_b = Set()
            for e in b
                push!(permuted_b, perm[e])
            end
            push!(current_b, permuted_b);
        end
        current_b = Set([Set(b) for b in current_b])
    if !(current_b in iso)
        push!(iso, current_b);
        counter = counter +1;
            #println(iso)
            #println(iso);
    end
    end 
    for m in iso
        push!(all_matroids, (matroid_from_bases(m, n_elements)))      
    end
   
end
N = length(all_matroids)
println(N)

37582


In [22]:
# check if a poset admits a rank function

# The poset p is encoded as a vector of elements that can be compared with <
# p must have a least element, stored as p[1]

# true if y covers x in p
covers(p, y, x) = x < y && !any(z -> x < z < y, p)

# the length of some maximal chain in p from p[1] to x
getrank(p, x) = x == p[1] ? 0 : getdist(p, p[1], x)

# the length of some maximal chain in p from x to y where x < y
function getdist(p, x, y)
    i = findfirst(z -> x < z < y, p)
    i === nothing ? 1 : getdist(p, x, p[i]) + getdist(p, p[i], y)
end

# true if p has a rank function
function hasrank(p)
    r = map(x -> getrank(p, x), p)
    for (i, x) in enumerate(p), (j, y) in enumerate(p)
         !covers(p, y, x) || r[j] == r[i]+1 || return [i, j, r[i], r[j], covers(p, y, x)]
    end
    true
end

# example: the poset of all subsets of some set

using Random: randperm
using SmallCollections: SmallBitSet, subsets

const S = SmallBitSet{UInt32}

N = 11
q = subsets(N) # all subsets of 1:N in colexicographic ordering
r = randperm(length(q))
println(q[3])
# random permutation of all subsets of 1:N, but with the empty set as first element
p = S[i == 1 || r[i] == 1 ? q[i] : q[r[i]] for i in 1:length(q)]
# for sets, `<` means proper subset


SmallBitSet{UInt64}([2])


2048-element Vector{SmallBitSet{UInt32}}:
 SmallBitSet([])
 SmallBitSet([1, 2, 5, 9, 11])
 SmallBitSet([1, 4, 5, 7, 10])
 SmallBitSet([1, 5, 7, 10, 11])
 SmallBitSet([2, 4, 5, 7, 8, 11])
 SmallBitSet([6, 8, 9, 10, 11])
 SmallBitSet([2, 3, 4, 5, 7, 11])
 SmallBitSet([1, 2, 7, 8, 9])
 SmallBitSet([2, 3, 5, 8])
 SmallBitSet([1, 4, 8, 9, 10])
 SmallBitSet([1, 2, 4, 6, 7, 8, 9, 10])
 SmallBitSet([1, 6, 11])
 SmallBitSet([2, 9, 10, 11])
 ⋮
 SmallBitSet([1, 2, 3, 5, 10])
 SmallBitSet([1, 4, 6, 7, 8, 10, 11])
 SmallBitSet([1, 3, 8, 9, 10])
 SmallBitSet([2, 4, 8, 9, 11])
 SmallBitSet([3, 5, 7])
 SmallBitSet([2, 3, 4])
 SmallBitSet([1, 2, 3, 9, 10, 11])
 SmallBitSet([2, 4, 6])
 SmallBitSet([3, 7, 8, 9, 10])
 SmallBitSet([2, 3, 6, 11])
 SmallBitSet([1, 2, 4, 7, 8])
 SmallBitSet([2, 6, 10, 11])

In [23]:

function el(b)
    possible_b = subsets(n_elements)
    index = -2
    for (l, x) in enumerate(possible_b)
        if x == SmallBitSet{UInt32}(b)
            index = l
        end
    end
    return index
end
println(el([2]))

3


In [25]:
subs= [SmallBitSet{UInt128}([])]
for matroid in all_matroids
    subset = SmallBitSet{UInt128}([])
    for basis in bases(matroid)
        a = el(basis)
        b = [a]
        b = SmallBitSet{UInt128}(b)
        subset = union(subset, b)
    end
    push!(subs, subset) # ????
end

hasrank(subs)

5-element Vector{Int64}:
 8658
 5298
   10
   10
    1